## Run AI reviewer for hypothetical examples at IT department


In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()  # Load variables from .env

from llm import create_client
from review import load_docs, perform_review, provide_feedback
import pandas as pd

# Create a real client
api_key = os.getenv("OPENAI_API_KEY")
client, model = create_client("gpt-4o-2024-05-13")

app_paths = ["24-001_backlog", "24-002_VMware", "24-003_Adobe"]
path = "docs/application/"
output_path = "docs/review/"

results = []
feedback_results= []
for application in app_paths:
    print(f"Review for {application} was started...")
    # Output format
    result = {}

    # Load application and related document
    file_path = path + application
    app_docs = load_docs(file_path)

    # Call the function
    review = perform_review(
        text=app_docs,
        model=model,
        client=client,
    )

    # Convert the dictionary to a string representation
    result["review_result"] = review
    table = pd.DataFrame.from_dict(review, orient='index')
    review_text = table.to_markdown()

    # Call the function
    feedback = provide_feedback(
        application_text=app_docs,
        review_text=review_text,
        model=model,
        client=client,
    )
    result["feedback_result"] = feedback

    output_file = f"review_{output_path}{application}.md"
    # Save as a properly formatted Markdown file
    with open(output_file, "w", encoding="utf-8") as f:
        f.write(f"# Review Result for Application ID: {application}\n\n")
        
        if sum((table["Decision"] != "Pass")) == 0:
            f.write("✅ **Application was approved.**\n\n")
        else:
            f.write("❌ **Application was rejected.**\n\n")
        
        f.write("## Criteria Review Result\n\n")
        f.write(table.to_markdown())
        f.write("```\n\n")

        f.write("## Feedback\n\n")
        f.write(feedback + "\n")

    print(f"Review for {application} was finished!")

# Print the result
print("Review was completed")

Using OpenAI API with model gpt-4o-2024-05-13.
Review was completed
